# Notebook use to analyse one specific file.

You can click `shift` + `enter` to run one cell, you can also click run in top menu.
To run all the cells, you can click `kernel` and `Restart and run all` in the top menu.

In [ ]:
import time
tp1 = time.time()

In [ ]:
# Some magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
# Ignore warnings in notebook
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Matplotlib to plot the data
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as patches
plt.rcParams['figure.figsize'] = 8,8
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
# Some module needed in the notebook
import numpy as np
import javabridge
import bioformats
from itkwidgets import view
from sklearn.externals import joblib

### The following path should direct to the folder "utils", on Window env it should have slash " / " and not backslash " \ " . 

In [ ]:
# Create a temporary python PATH to the module that we are using for the analysis
import sys
sys.path.insert(0, "/Users/Espenel/Desktop/Mini-Grant-Image-analysis/2018/Chloe/ChromosomeDetectionChloe/utils")

In [ ]:
from chromosome_dsb import *

In [ ]:
# Need to create a javabridge to use bioformats to open proprietary format
javabridge.start_vm(class_path=bioformats.JARS)

### In the path_data variable you should enter the path to your data:

In [ ]:
path_data = '/Users/Espenel/Desktop/Mini-Grant-Image-analysis/2018/Chloe/data_chloe/test_batch/exp1/'

In [ ]:
position, time_point = load_data.stage_position(path_data)

# Set Parameters

In [ ]:
# Size kernel for background substraction, should be a little larger than the object of interest
back_sub_FOCI = 5
back_sub_Nucleus = 20
# LOCI detection:
# Smallest object (in pixels) to be detected
smaller = 1
# Largest object to be detected
largest = 5
# Threshold above which to look for
threshold = 12000    

# Find "Skeleton" of gonad

In [ ]:
skelete = load_data.skeleton_coord(position,time_point)

# Load Image

### In the path_img you can enter the name of your specific image "/....dv"

In [ ]:
path_img = path_data + '/2017-04-12_RAD51-HTP3_cku80-exo1_002_visit_13_D3D_ALX.dv'

In [ ]:
image, meta, directory = load_data.load_bioformats(path_img)

# Plot "Skeleton" of gonad

In [ ]:
data = np.concatenate((position,time_point[:, np.newaxis]), axis=1)
sort_data = data[np.argsort(data[:,2])]

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.scatter(skelete[:,0], skelete[:,1], s=0.5)
stage_pos = ax.scatter(sort_data[:,0], sort_data[:,1])
working_on = ax.scatter(meta["PositionX"], meta["PositionY"], s=300, color = "r")
plt.legend([stage_pos, working_on], ["Stage Positions", 
                                     "Image currently working on"],
                                      loc=0,fontsize='large')

In [ ]:
img = image[:,:,:,3]

### Optionally you can visualyze your data

In [ ]:
#view(visualization.convert_view(img))

# Find the nucleus in the image

### First need to load the classifier (clf) and scaler.

In [ ]:
clf = joblib.load("/Users/Espenel/Desktop/Mini-Grant-Image-analysis/2018/Chloe/ChromosomeDetectionChloe/clf_scaler/clf")
scaler = joblib.load("/Users/Espenel/Desktop/Mini-Grant-Image-analysis/2018/Chloe/ChromosomeDetectionChloe/clf_scaler/scaler")

In [ ]:
tp_1 = time.time()
result = search.rolling_window(img, clf, scaler)
tp_2 = time.time()
print(tp_2-tp_1)

In [ ]:
bbox_ML = search.non_max_suppression(result, probaThresh=0.8, overlapThresh=0.3)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(np.amax(img,axis=0), vmax=img.max()/2, alpha = 0.8)
for coord in bbox_ML:
    circles1 = patches.Circle((coord[0]+35,coord[1]+35),30, linewidth=3,edgecolor='r',facecolor='none')
    ax.add_patch(circles1)

# Background Substraction

In [ ]:
FOCI_ch, _ = img_analysis.background_correct(image, ch=1, size=back_sub_FOCI)
Nucleus_ch, _ = img_analysis.background_correct(image, ch=3, size=back_sub_Nucleus)

In [ ]:
visualization.plot_background(image, FOCI_ch, Nucleus_ch)

# Finding the Blobs/FOCI

In [ ]:
blobs = img_analysis.find_blob(FOCI_ch, meta, directory, smaller = smaller, 
                               largest = largest, thresh = threshold,
                               plot=True)

# Binarization of the Channel with nucleus

In [ ]:
binary = img_analysis.binarization(Nucleus_ch)

### Optionaly, you can visualyze the result of the binarization

In [ ]:
#view(visualization.convert_view(binary))

# Load the position of the different nucleus

In [ ]:
#bbox_ML = np.load("/Users/Espenel/Desktop/Mini-Grant-Image-analysis/2018/Chloe/13/bbox_3D.npy")

# Mask FOCI that are not on the nucleus

In [ ]:
masked = search.find_foci(blobs, FOCI_ch, Nucleus_ch, binary, bbox_ML)

# Mask FOCI that are not on a nucleus found by the Machine Learning

In [ ]:
res, bb_mask = search.binary_select_foci(bbox_ML, Nucleus_ch, masked)

# Find and remove FOCI that were counted twice

In [ ]:
num, cts, dup_idx, mask = search.find_duplicate(res, bb_mask)

In [ ]:
visualization.plot_result(img, res, bbox_ML,\
                          cts, num, meta, directory, save = False)

In [ ]:
dist_tip = img_analysis.distance_to_tip(bbox_ML, skelete, meta)

In [ ]:
chro_pos = np.squeeze(np.dstack((bbox_ML[:,0]+35,
                          bbox_ML[:,1]+35, bbox_ML[:,4])))

In [ ]:
df = img_analysis.final_table(meta, bbox_ML,  \
                         dist_tip, cts, num, \
                         directory, save = False)

In [ ]:
df

In [ ]:
to_save = {'back_sub_ch1' : back_sub_ch1,
           'back_sub_ch2' : back_sub_ch2,
           'back_sub_ch3' : back_sub_ch3,
           'small_object' : smaller,
           'large_object' : largest,
           'threshold' : thresh}

log.log_file(directory, meta, **to_save)

In [ ]:
tp2 = time.time()

In [ ]:
print("It took {}sec".format(int(tp2-tp1)))